## Importing Packages

In [1]:
## Importing Packages
import subprocess as sp
import os, sys
import shutil

## Executing terminal command in terminal from python

In [2]:
### Executing terminal command in terminal from python

def execute_command(exec_command, msg=True):
    '''
    To Execute Command in terminal
        Input: 'ls -lsha'
        Transformation: ['ls', '-lsha']
        Output: Output of the command, and process completion in terminal
    
    subprocess.run(args, *, stdin=None, input=None, stdout=None, stderr=None, 
                shell=False, cwd=None, timeout=None, check=False, encoding=None, 
                errors=None, env=None)
    '''
    
    if msg is True: print('\nExecuting Command "{}"'.format(exec_command))
    try:
        exec_ter = sp.run(exec_command.split(' '), stdout=sp.PIPE, stderr=sp.PIPE)
        task_status = 'Completed' if exec_ter.returncode == 0 else 'CalledProcessError' 
        if msg is True: print('Command ' + ('sucessfully executed' if task_status == 'Completed' else 'failed to be executed'))
        txt_enc = exec_ter.stdout
        exec_msg = txt_enc.decode('ascii')
        if (msg is True) & (len(exec_msg)>0):print('Output:\n'+'-'*100+'\n|\t'+exec_msg.replace('\n','\n|\t')+'\n'+'-'*100)
        if task_status  is 'Completed':
            return exec_msg
        else:
            return 'TaskFailed'
    
    except Exception as e:
        print('-'*50)
        print("Exception: {}".format(e))#e.errno, e.strerror))
        print("Unexpected error:", sys.exc_info()[0])
        print('-'*50+'\n')
        return 'TaskFailed'

trial_command_to_exec = [
    'ls -lsha', 
    'cd ..', ## will have an error
    'ls -lsha'
]

for command in trial_command_to_exec:
    execute_command(command)
    


Executing Command "ls -lsha"
Command sucessfully executed
Output:
----------------------------------------------------------------------------------------------------
|	total 48K
|	4.0K drwxrwxr-x  4 ubuntu ubuntu 4.0K Jun 18 14:55 .
|	4.0K drwxr-xr-x 32 ubuntu ubuntu 4.0K Jun 18 14:39 ..
|	 24K -rwxrwxrwx  1 ubuntu ubuntu  22K Jun 18 14:55 Development.ipynb
|	4.0K drwxrwxr-x  8 ubuntu ubuntu 4.0K Jun 18 14:39 .git
|	4.0K drwxrwxr-x  2 ubuntu ubuntu 4.0K Jun 18 14:42 .ipynb_checkpoints
|	4.0K -rw-rw-r--  1 ubuntu ubuntu   69 Jun 18 14:39 README.md
|	4.0K -rwxrwxrwx  1 ubuntu ubuntu  126 Jun 18 14:41 repo_link_input.txt
|	
----------------------------------------------------------------------------------------------------

Executing Command "cd .."
--------------------------------------------------
Exception: [Errno 2] No such file or directory: 'cd': 'cd'
Unexpected error: <class 'FileNotFoundError'>
--------------------------------------------------


Executing Command "ls -lsha"
Com

## To convert Git Clone command such that password can also be fed directly

In [3]:
''' ## Feeding Password with the username in the git ## Also encoding password
https://stackoverflow.com/questions/10054318/how-do-i-provide-a-username-and-password-when-running-git-clone-gitremote-git/55721882#55721882
'''

git_username = 'username'
git_passsword = 'password'


# https://www.urlencoder.org/
special_char_dict = {
    '%':'%25', ## caution with this
    '.':'.',
    '+':'%2B',
    '@':'%40',
    '#':'%23',
    '!':'%21',
    '$':'%24',
    '^':'%5E',
    '&':'%26',
    '*':'%2A',
    '_':'_',
    '-':'-'    
}
upd_user = ''.join([ special_char_dict[char] if char in special_char_dict else char for char in git_username ])
upd_pass = ''.join([ special_char_dict[char] if char in special_char_dict else char for char in git_passsword ])

sample_cmd = 'git clone https://github.com/MR901/AIMA_AdversarialSearch.git'
sample_cmd_to_exec = sample_cmd.replace(git_username, upd_user+':'+upd_pass)
sample_cmd_to_exec

'git clone https://github.com/MR901/AIMA_AdversarialSearch.git'

## Read the file containg "git clone links"

In [4]:
### Read the file contaiing "git clone links"

file_containing_git_clone_link_name =  'repo_link_input.txt'

curr_dir_name = execute_command('pwd').split('\n')[0]
input_file_name = curr_dir_name + '/' + file_containing_git_clone_link_name

f= open( input_file_name, 'r')
txt = f.read()
f.close()

git_clone_cmd = txt.splitlines() # txt.split('\n')
git_cmd_to_exec = [ cmd.replace(git_username, upd_user+':'+upd_pass) for cmd in git_clone_cmd ] 

## Taking link and cleaning these too
temp_dict = {}
for ele in git_cmd_to_exec:
    if ele in temp_dict:
        temp_dict[ele] += 1
    else:
        temp_dict[ele] = 1

git_cmd_to_exec = [ ele for ele in temp_dict.keys() if 'git' in ele ] 

print(git_cmd_to_exec[0:5])


Executing Command "pwd"
Command sucessfully executed
Output:
----------------------------------------------------------------------------------------------------
|	/home/ubuntu/Documents/PersonalWorkSpace/MyGits/interaction_with_git
|	
----------------------------------------------------------------------------------------------------
['git clone https://github.com/MR901/interaction_with_git.git', 'git clone https://github.com/MR901/AIMA_AdversarialSearch.git']


### Getting all the available branches in the repo and and create those folder and adjust the content accordingly

In [5]:
def clone_repo_and_create_folder_for_branch(transf_git_clone_cmd, path_where_to_create = None):
    '''
    Input: Git Clone Command after special character in username and passsword has been Transformed
    Output: 
        current folder ---+--> git folder1 +--> repo_branch1 +--> contents
                          |                |--> repo_branch2 +--> contents
                          |                |--> repo_branch3 +--> contents
                          +--> git folder2 +--> repo_branch1 +--> contents
                                           |--> repo_branch2 +--> contents
                                           |--> repo_branch3 +--> contents
    '''

    ## creating a new/repo folder based on git name and going inside it 
    if path_where_to_create is not None:
        if os.path.exists(path_where_to_create) is False:
            print('Path has been created :', path_where_to_create)
            os.mkdir(path_where_to_create)
        os.chdir(path_where_to_create)
    orig_dir = os.getcwd() #execute_command('pwd').split('\n')[0]+'/'
    git_folder_name = transf_git_clone_cmd.split('/')[-1].split('.')[0]
    os.mkdir(git_folder_name)
    git_dir = orig_dir +'/'+git_folder_name
    os.chdir(git_dir)
    print('\nNew directory "{}" is created. \nCurrent working directory path is "{}"'.format(git_folder_name, git_dir))

    ## Cloning a repository and changing the w.d. to repo
    content_prev = execute_command('ls',msg='off').split('\n')
    task_msg = execute_command(transf_git_clone_cmd)
    if task_msg is 'TaskFailed': print('CloneCmdFailed'); return 'CloneCmdFailed'

    repo_folder_name = [ con for con in execute_command('ls',msg='off').split('\n') if con not in content_prev ][0]
    repo_dir = os.getcwd() +'/'+ repo_folder_name
    os.chdir(repo_dir)
    print('\nNew directory "{}" is created. \Current working directory path is "{}"'.format(repo_folder_name, repo_dir))

    ## Getting all the branch name
    all_branches = execute_command('git branch -a').split('\n')
    all_branches = [ branch.split('/')[-1] for branch in all_branches if 'remotes/origin/' in branch and '->' not in branch ]
    print('All the branches that are present : ', all_branches)

    ## Creating folders wrt to the branch names
    os.chdir(git_dir)
    for i in range(len(all_branches)):
        print('\n'+str(i)+'\tCurrently working on the branch :', all_branches[i])
        repo_branch_name = repo_folder_name + '__br-' + all_branches[i]
        if i+1 < len(all_branches):
            print('\tCreating a repo branch folder and pasting the content inside it. Also changing the branch inside it.')
            os.mkdir(repo_branch_name)
            execute_command('cp -r {0}/. {1}/.'.format(repo_folder_name, repo_branch_name))
        else:
            print('\tRenaming the original repo folder name and changing the branch inside it.')
            os.rename(repo_folder_name, repo_branch_name)

    ## Changing the branches in each individual folder respectively
    all_dir = os.listdir() ## current dir = git_dir
    for wd in all_dir:
        # change wd to the branch folder
        os.chdir(git_dir+'/'+wd)
        # change the branch
        execute_command('git checkout {}'.format(wd.split('__br-')[-1]))
    #     # change the wd back to actual ## no need since using absolute path
    #     os.chdir(git_dir)

    ## Changing the directory back to Original Dir
    os.chdir(orig_dir)

    '''
    # ## Removing all the content from this directory <--- to independently create folder for each branch
    # def delete_file_or_folder(path):
    #     if os.path.isfile(path):
    #         os.unlink(path) # os.remove(path)
    #     else:
    #         # os.rmdir(path) # for empty dir
    #         shutil.rmtree(path) ## to delete all files inside the ddirectory too
    # all_files = [ f for f in execute_command('ls -a').split('\n') if f not in ['','.','..'] ]
    # for path in all_files:
    #     delete_file_or_folder(path)
    # all_files = [ f for f in execute_command('ls -a','off').split('\n') if f not in ['','.','..'] ]
    # if len(all_files)<=3: print('Empty Directory : Directory has been cleaned fully.')
    '''

    return 'CloneCmdCompleted'

## Sample Test
download_where = 'SS_GitRepos_SampleBackup'
# if os.path.exists(download_where):
clone_repo_and_create_folder_for_branch(sample_cmd_to_exec, download_where)
# else:
#     print("path doesn't exist")

Path has been created : SS_GitRepos_SampleBackup

New directory "AIMA_AdversarialSearch" is created. 
Current working directory path is "/home/ubuntu/Documents/PersonalWorkSpace/MyGits/interaction_with_git/SS_GitRepos_SampleBackup/AIMA_AdversarialSearch"

Executing Command "git clone https://github.com/MR901/AIMA_AdversarialSearch.git"
Command sucessfully executed

New directory "AIMA_AdversarialSearch" is created. \Current working directory path is "/home/ubuntu/Documents/PersonalWorkSpace/MyGits/interaction_with_git/SS_GitRepos_SampleBackup/AIMA_AdversarialSearch/AIMA_AdversarialSearch"

Executing Command "git branch -a"
Command sucessfully executed
Output:
----------------------------------------------------------------------------------------------------
|	* master
|	  remotes/origin/HEAD -> origin/master
|	  remotes/origin/master
|	
----------------------------------------------------------------------------------------------------
All the branches that are present :  ['master']



'CloneCmdCompleted'

In [6]:
# download_where = '/home/mohit/Documents/MyGits/Interaction_with_git/SS_GitRepos_Backup_20190508'
download_where = '/home/ubuntu/Documents/PersonalWorkSpace/MyGits/interaction_with_git/GitRepos_Backup_20190508'
clone_cmd_failed, clone_cmd_success = [], []
for cmd in git_cmd_to_exec:
    try:
        msg = clone_repo_and_create_folder_for_branch(cmd, download_where)
        if msg is 'CloneCmdCompleted':
            clone_cmd_success.append(cmd)
        elif msg is 'CloneCmdFailed':
            clone_cmd_failed.append(cmd)
    except:
        clone_cmd_failed.append(cmd)

Path has been created : /home/ubuntu/Documents/PersonalWorkSpace/MyGits/interaction_with_git/GitRepos_Backup_20190508

New directory "interaction_with_git" is created. 
Current working directory path is "/home/ubuntu/Documents/PersonalWorkSpace/MyGits/interaction_with_git/GitRepos_Backup_20190508/interaction_with_git"

Executing Command "git clone https://github.com/MR901/interaction_with_git.git"
Command sucessfully executed

New directory "interaction_with_git" is created. \Current working directory path is "/home/ubuntu/Documents/PersonalWorkSpace/MyGits/interaction_with_git/GitRepos_Backup_20190508/interaction_with_git/interaction_with_git"

Executing Command "git branch -a"
Command sucessfully executed
Output:
----------------------------------------------------------------------------------------------------
|	* master
|	  remotes/origin/HEAD -> origin/master
|	  remotes/origin/master
|	
---------------------------------------------------------------------------------------------

In [7]:
clone_cmd_success

['git clone https://github.com/MR901/interaction_with_git.git',
 'git clone https://github.com/MR901/AIMA_AdversarialSearch.git']

In [8]:
clone_cmd_failed

[]